This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_residuals_histogram(y_true, y_pred, savepath,
                             stats, title='residuals histogram', label='residuals'):
    """
    Method to calculate and plot the histogram of residuals from regression model

    Args:

        y_true: (numpy array), array of true y data values

        y_pred: (numpy array), array of predicted y data values

        savepath: (str), path to save the plotted precision-recall curve

        stats: (dict), dict of training or testing statistics for a particular run

        title: (str), title of residuals histogram

        label: (str), label used for axis labeling

    Returns:

        None

    """

    # make fig and ax, use x_align when placing text so things don't overlap
    x_align = 0.64
    fig, ax = make_fig_ax(x_align=x_align)

    #ax.set_title(title)
    # do the actual plotting
    residuals = y_true - y_pred

    #Output residuals data and stats to spreadsheet
    path = os.path.dirname(savepath)
    pd.DataFrame(residuals).describe().to_csv(os.path.join(path,'residual_statistics.csv'))
    pd.DataFrame(residuals).to_csv(path+'/'+'residuals.csv')

    #Get num_bins using smarter method
    num_bins = get_histogram_bins(y_df=residuals)
    ax.hist(residuals, bins=num_bins, color='b', edgecolor='k')

    # normal text stuff
    ax.set_xlabel('Value of '+label, fontsize=16)
    ax.set_ylabel('Number of occurences', fontsize=16)

    # make y axis ints, because it is discrete
    #ax.yaxis.set_major_locator(MaxNLocator(integer=True))

    plot_stats(fig, stats, x_align=x_align, y_align=0.90)
    plot_stats(fig, pd.DataFrame(residuals).describe().to_dict()[0], x_align=x_align, y_align=0.60)

    fig.savefig(savepath, dpi=DPI, bbox_inches='tight')


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
y_true = array([1.50371026e+02, 2.22927321e+02, 2.29790128e+02, 2.41537703e+02,
       4.07147186e+02, 3.46912901e+02, 2.44558934e+02, 7.98843255e+01,
       9.96717956e+01, 1.20156284e+02, 1.06566685e+02, 1.86835775e+02,
       7.75902143e+01, 1.78590946e+02, 1.02665208e+02, 1.27306711e+02,
       1.47491884e+02, 1.01679243e+02, 1.70112526e+02, 1.71679718e+02,
       1.44367703e+02, 1.90002071e+02, 1.90460889e+02, 1.25776102e+02,
       1.72040539e+02, 1.85728826e+02, 1.37488659e+02, 7.23673571e+01,
       1.74631224e+02, 3.79332462e+01, 9.24493164e+01, 0.00000000e+00,
       9.78662706e+01, 9.51442246e+01, 2.36177315e+02, 1.08666855e+02,
       7.09734216e+01, 2.91384616e+02, 6.60766101e+01, 2.36664682e+02,
       4.08488586e+02, 5.15805330e+02, 1.22389663e+02, 8.81149314e+01,
       1.57715585e+02, 2.37521275e+02, 1.28364345e+02, 2.38334229e+02,
       3.47383100e+02, 4.16012879e+01, 9.79676617e+00, 7.48766194e+00,
       0.00000000e+00, 1.56215250e+01, 6.31512971e+01, 0.00000000e+00,
       5.64799569e+01, 1.49611167e+02, 0.00000000e+00, 0.00000000e+00,
       3.11290486e+01, 7.93342932e+01, 5.04901919e+01, 0.00000000e+00,
       2.12302521e+02, 1.98106664e+02, 1.18302658e+02, 3.77502009e+02,
       1.19774983e+02, 5.00254523e+01, 0.00000000e+00, 3.31384554e+01,
       4.09146349e+01, 3.93315709e+01, 4.89115430e+01, 0.00000000e+00,
       0.00000000e+00, 4.51430137e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.69769380e+02, 1.08224668e+02, 3.34361123e+01,
       4.06187327e+01, 6.70851015e+01, 4.33637207e+01, 3.81461657e+01,
       3.21439514e+01, 3.92412752e+01, 2.10808929e+02, 2.32209209e+02,
       1.57374363e+02, 0.00000000e+00, 1.55792869e+02, 9.57636772e+01,
       0.00000000e+00, 0.00000000e+00, 6.92693877e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.06802766e+01,
       5.53782777e+01, 4.07973166e+01, 5.08263675e+01, 1.41470544e+02,
       2.39414173e+01, 7.92325765e+01, 6.53557069e+01, 3.98312067e+01,
       2.18803383e+01, 4.48563810e+01, 1.33902387e+01, 3.61795100e+01,
       1.44194572e+02, 1.07975067e+02, 1.12573845e+02, 9.85807552e+01,
       0.00000000e+00, 2.83906453e+02, 2.02289181e-01, 1.51553671e+02,
       5.24822446e+01, 8.98167378e+01, 8.77843631e+01, 9.36717096e+01,
       0.00000000e+00, 8.69178900e+00, 4.36717688e+00, 1.27378189e+02,
       1.71956703e+02, 2.58665310e+02, 0.00000000e+00, 4.21689745e+01,
       3.15202492e+01, 1.26777344e+02, 2.29064683e+02, 1.99684569e+02,
       0.00000000e+00, 1.38924377e+01, 5.14231024e+01, 0.00000000e+00,
       0.00000000e+00, 7.81817223e+01, 4.55261914e+01, 4.79204659e+01,
       0.00000000e+00, 5.09486685e+01, 6.98113911e+01, 1.55465078e+02,
       1.32773863e+02, 2.04099993e+02, 6.77439450e+01, 7.83502295e+01,
       7.67577252e+01, 6.94770317e+01, 4.42032278e+01, 3.74738040e+01,
       5.46661575e+01, 5.67330262e+01, 0.00000000e+00, 0.00000000e+00,
       3.95429854e+01, 2.82496436e+01, 0.00000000e+00, 3.42510607e+01,
       1.62892176e+02, 1.50562981e+02, 0.00000000e+00, 1.41471166e+02,
       1.95442993e+02, 2.85835223e+01, 1.15904985e+01, 5.50100622e+01,
       7.11848257e+01, 1.03266329e+02, 6.89092871e+01, 0.00000000e+00,
       3.69821609e+00, 5.31766390e-01, 1.12780820e+01, 0.00000000e+00,
       0.00000000e+00, 4.45467226e+01, 7.37535702e+01, 0.00000000e+00,
       2.44348237e+02, 1.36431444e+02, 1.85197855e+02, 2.02269808e+02,
       2.65270592e+02, 1.32871922e+02, 3.78453371e+02, 8.83272115e+01,
       9.03169483e+01, 9.77903794e+01, 2.41664953e+02, 1.81295285e+02,
       2.49724729e+02, 2.30118336e+02, 1.14407716e+02, 1.31761462e+02,
       1.89952208e+02, 1.24477908e+02, 1.26700822e+02, 1.49948333e+02,
       1.55694637e+02, 1.50672468e+02, 1.63799175e+02, 2.18442616e+02,
       9.81125625e+01, 1.52280199e+02, 1.91836674e+02, 2.12302508e+02,
       1.78586168e+02, 1.01768108e+02, 1.56136057e+02, 1.35390845e+02,
       1.69573331e+02, 2.73613179e+02, 2.37992715e+02, 3.09856621e+02,
       2.04046571e+02, 2.55478983e+02, 3.49453381e+02, 3.18399908e+02,
       7.64960376e+01, 2.05564030e+02, 7.04378967e+01, 8.23163755e+01,
       9.77091114e+01, 8.35495499e+01, 2.08659157e+02, 1.77317832e+02,
       2.80854292e+02, 2.06610134e+02, 1.54641811e+02, 1.00117427e+02,
       1.40265182e+02, 9.48376021e+01, 9.54547321e+01, 1.83449190e+02,
       2.13333228e+02, 1.08425980e+02, 1.59401405e+02, 1.85807040e+02,
       2.40576558e+02, 2.59216571e+02, 3.15514875e+02, 1.44889042e+02,
       2.38988178e+02, 2.52611496e+02, 2.18137285e+02, 3.05414508e+02,
       7.10448978e+01, 7.89063338e+01, 6.56442278e+01, 1.70005981e+02,
       1.38575932e+02, 1.79659313e+02, 1.18314107e+02, 1.23035360e+02,
       1.36443088e+02, 5.95241562e+01, 1.29155595e+02, 5.68526124e+01,
       1.65456460e+02, 3.12115625e+02, 2.42965465e+02, 1.91404536e+02,
       2.00760493e+02, 9.56831956e+02, 1.28639131e+02, 1.23367070e+02,
       1.38844996e+02, 5.74165434e+01, 1.04682468e+02, 1.10065846e+02,
       1.43722366e+02, 1.09913097e+02, 1.94691060e+02, 1.15142225e+02,
       1.51089123e+02, 9.45208382e+01, 1.26955312e+02, 1.78159873e+02,
       1.56518089e+02, 9.04495542e+01, 6.89800768e+01, 2.66189733e+01,
       5.18182329e+01, 3.70668895e+01, 1.24273117e+02, 2.42317220e+01,
       3.26473248e+01, 7.34595746e+01, 7.19683460e+01, 4.18854297e+01,
       4.30159281e+01, 7.21745594e+01, 4.85930390e+01, 9.42635786e+01,
       4.29043523e+01, 1.73730028e+02, 1.76433607e+02, 2.10616630e+02,
       1.60456727e+02, 1.67324805e+02, 2.11267960e+02, 1.87207178e+02,
       2.36861914e+02, 1.33312148e+02, 1.54672899e+02, 2.00391998e+02,
       9.14620807e+01, 1.00815664e+02, 2.71319339e+02, 1.37479653e+02,
       1.10285836e+02, 1.35954460e+02, 1.83271284e+02, 1.17247059e+02,
       1.37791901e+02, 1.56027039e+02, 1.35955483e+02, 2.16122195e+02,
       1.36808521e+02, 1.35369317e+02, 1.21692419e+02, 9.37481720e+01,
       7.98648319e+01, 1.34354983e+02, 3.33198729e+01, 2.83093128e+02,
       2.23792303e+01, 1.09582964e+02, 9.88464407e+01, 1.40488835e+02,
       1.11369551e+02, 1.92009283e+02, 1.04314376e+01, 9.43179777e+01,
       1.32019981e+02, 1.78944559e+02, 9.04350450e+01, 0.00000000e+00,
       1.55909864e+02, 9.38423029e+01, 7.18885960e+01, 9.37906658e+01,
       5.63388158e+01, 4.54043655e+01, 6.24481656e+01, 0.00000000e+00,
       1.01755733e+02, 2.83147240e+01, 2.55044144e+00, 6.11195316e+01,
       2.86637589e+01, 7.99376675e+00, 0.00000000e+00, 3.23684503e+01,
       1.20026452e+02, 3.86517633e+01, 2.77992144e+01, 1.72121127e+01,
       0.00000000e+00, 0.00000000e+00, 1.84387405e+02, 7.65032664e+01,
       6.74148765e+01, 6.23324679e+01, 3.60966810e+01, 9.16707430e+01,
       1.16752352e+02, 4.08206874e+01, 4.60103452e+01, 3.88036349e+01,
       8.61157898e+01, 8.42613186e+01, 8.65192126e+01, 5.33006835e+01,
       6.76609259e+01, 1.05624926e+02, 4.11379073e+01, 2.72516127e+01,
       4.63580983e+01, 3.15042600e+01, 3.05960388e+01, 4.59371875e+01,
       2.93690973e+01, 9.40057842e+01, 4.67667976e+01, 1.20061041e+02,
       2.84167104e+01, 4.54076116e+01, 4.53272821e+01, 5.00275181e+01,
       8.97135102e+01, 1.25245007e+02, 7.00096850e+00, 7.80670028e+01,
       7.76829500e+01, 1.34709235e+01, 5.97432500e+00, 7.83039861e+01,
       2.19120080e+01, 7.17549475e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 8.66887169e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00])
y_pred = array([0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 4.49083689e+001, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       5.91228234e-199, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       3.50436623e-199, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       2.20687289e+001, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000])
savepath = '/home/nerve/Desktop/skunkworks/lane_schultz/hyper_parameters/test/DoNothing/DoNothing/KernelRidge_DoNothing_DoNothing_GridSearch/RepeatedKFold/split_0/test_residuals_histogram.png'
stats = OrderedDict([('R2', 9.033071994635211e-05), ('root_mean_squared_error', 143.50339594602403), ('mean_absolute_error', 106.38736660129737), ('rmse_over_stdev', 1.5076078525636898)])
title = 'test_residuals_histogram'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_residuals_histogram(y_true, y_pred, savepath, stats, title, label)
display(Image(filename='test_residuals_histogram.png'))
